<a href="https://colab.research.google.com/github/shubham620/cdti_jaipur_project/blob/main/cdti_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import pandas as pd
import re


In [100]:
# Load CSV file
df = pd.read_csv("demo_input_data_3.csv")

# Display the raw data
print("RAW INPUT DATA:")
df.head()


RAW INPUT DATA:


,id,text,source,time,location
0,1_1,Crowd gathering reported near City Mall,social,9:00:00 AM,"City Mall, Jaipur"
1,1_2,Police received call regarding crowd near City...,emergency,9:04:00 AM,"City Mall, Jaipur"
2,2_1,Fire and smoke reported near Railway Station,social,9:10:00 AM,"Railway Station, Jaipur"
3,2_2,Fire brigade dispatched to Railway Station,official,9:14:00 AM,"Railway Station, Jaipur"
4,3_1,Medical emergency reported at Metro Station,social,9:20:00 AM,"Metro Station, Jaipur"


In [101]:
def clean_text(text):
    text = text.lower()                     # convert to lowercase
    text = re.sub(r"[^a-zA-Z\s]", "", text) # remove special characters
    text = re.sub(r"\s+", " ", text)        # remove extra spaces
    return text.strip()


In [102]:
# Apply text cleaning
df["clean_text"] = df["text"].apply(clean_text)

# Show cleaned text
print("CLEANED DATA:")
df[["id", "text", "clean_text"]]


CLEANED DATA:


,id,text,clean_text
0,1_1,Crowd gathering reported near City Mall,crowd gathering reported near city mall
1,1_2,Police received call regarding crowd near City...,police received call regarding crowd near city...
2,2_1,Fire and smoke reported near Railway Station,fire and smoke reported near railway station
3,2_2,Fire brigade dispatched to Railway Station,fire brigade dispatched to railway station
4,3_1,Medical emergency reported at Metro Station,medical emergency reported at metro station
...,...,...,...
195,98_2,Electric department restoring supply,electric department restoring supply
196,99_1,Fire reported near Civil Lines warehouse,fire reported near civil lines warehouse
197,99_2,Fire brigade responding,fire brigade responding
198,100_1,Traffic disruption reported near NH-21 toll,traffic disruption reported near nh toll


In [103]:
!pip install sentence-transformers


In [104]:
from sentence_transformers import SentenceTransformer
import numpy as np


In [105]:
# Load a lightweight sentence embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")


In [106]:
# Convert cleaned text into embeddings (numeric meaning)
embeddings = model.encode(df["clean_text"].tolist())

# Check embedding shape
print("Embedding shape:", embeddings.shape)


Embedding shape: (200, 384)


In [107]:
# Store embeddings inside dataframe
df["embedding"] = embeddings.tolist()

df[["id", "clean_text"]].head()


,id,clean_text
0,1_1,crowd gathering reported near city mall
1,1_2,police received call regarding crowd near city...
2,2_1,fire and smoke reported near railway station
3,2_2,fire brigade dispatched to railway station
4,3_1,medical emergency reported at metro station


In [108]:
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity


In [109]:
# Convert list of embeddings to numpy array
X = np.array(df["embedding"].tolist())

# Compute cosine similarity between all messages
similarity_matrix = cosine_similarity(X)


In [110]:
# Convert similarity to distance
distance_matrix = 1 - similarity_matrix

# Fix floating point negatives (important)
distance_matrix[distance_matrix < 0] = 0



In [111]:
# Apply DBSCAN clustering
dbscan = DBSCAN(
    eps=0.3,          # similarity threshold (tunable)
    min_samples=2,    # minimum messages to form an event
    metric="precomputed"
)

df["event_id"] = dbscan.fit_predict(distance_matrix)


In [112]:
# Display messages grouped by event
for event in df["event_id"].unique():
    print(f"\nEVENT {event}:")
    display(df[df["event_id"] == event][["id", "text", "source", "location"]])



EVENT 0:


,id,text,source,location
0,1_1,Crowd gathering reported near City Mall,social,"City Mall, Jaipur"
1,1_2,Police received call regarding crowd near City...,emergency,"City Mall, Jaipur"
16,9_1,Crowd forming near Airport Road checkpoint,social,"Airport Road, Jaipur"
28,15_1,Public gathering reported near Metro Station,social,"Metro Station, Jaipur"
29,15_2,Authorities monitoring crowd at Metro Station,news,"Metro Station, Jaipur"
40,21_1,Crowd gathering reported near City Mall parking,social,"City Mall, Jaipur"
41,21_2,Police monitoring crowd near City Mall parking,official,"City Mall, Jaipur"
56,29_1,Crowd forming near Metro Station entrance,social,"Metro Station, Jaipur"
68,35_1,Public gathering reported near Airport Road,social,"Airport Road, Jaipur"
69,35_2,Authorities monitoring crowd near Airport Road,news,"Airport Road, Jaipur"



EVENT 1:


,id,text,source,location
2,2_1,Fire and smoke reported near Railway Station,social,"Railway Station, Jaipur"
3,2_2,Fire brigade dispatched to Railway Station,official,"Railway Station, Jaipur"
18,10_1,Fire reported in commercial building near Civi...,social,"Civil Lines, Jaipur"
19,10_2,Fire services responding to incident in Civil ...,emergency,"Civil Lines, Jaipur"
42,22_1,Fire alarm triggered near Railway Station plat...,social,"Railway Station, Jaipur"
43,22_2,Fire services inspecting alarm near Railway St...,official,"Railway Station, Jaipur"
58,30_1,Fire reported in residential area of Civil Lines,social,"Civil Lines, Jaipur"
59,30_2,Fire brigade responding to Civil Lines incident,emergency,"Civil Lines, Jaipur"
82,42_1,Fire alarm reported near Airport terminal,social,"Airport Road, Jaipur"
83,42_2,Fire services checking alarm near airport,official,"Airport Road, Jaipur"



EVENT 2:


...

,id,text,source,location
80,41_1,Traffic jam reported near Bus Stand,social,"Bus Stand, Jaipur"
114,58_1,Traffic jam reported near University Road,social,"University Road, Jaipur"



EVENT 14:


,id,text,source,location
86,44_1,Power outage reported in Mansarovar sector 5,social,"Mansarovar, Jaipur"
170,86_1,Power outage reported near Mansarovar sector 1,social,"Mansarovar, Jaipur"



EVENT 15:


,id,text,source,location
88,45_1,Unverified threat message circulating online,social,Jaipur
89,45_2,Authorities investigating online threat message,official,Jaipur


In [113]:
# Remove noise messages (event_id = -1)
events_df = df[df["event_id"] != -1]

# Group messages by event
event_summary = events_df.groupby("event_id").agg(
    message_count=("id", "count"),
    sources=("source", lambda x: list(set(x))),
    locations=("location", lambda x: list(set(x)))
).reset_index()

event_summary



,event_id,message_count,sources,locations
0,0,28,"[news, social, emergency, official]","[Airport Road, Jaipur, Railway Station, Jaipur..."
1,1,26,"[social, official, emergency]","[Airport Road, Jaipur, Bus Stand, Jaipur, Rail..."
2,2,44,"[social, emergency, official]","[Airport Road, Jaipur, Bus Stand, Jaipur, Rail..."
3,3,16,"[social, official]","[Metro Station, Jaipur, Railway Station, Jaipu..."
4,4,10,"[social, official]","[Airport Road, Jaipur, Bus Stand, Jaipur, Metr..."
5,5,12,"[news, social, official]","[University Road, Jaipur]"
6,6,7,[social],"[Metro Station, Jaipur, Bus Stand, Jaipur, Vai..."
7,7,13,[official],"[Bus Stand, Jaipur, Mansarovar, Jaipur, Univer..."
8,8,5,[official],"[Airport Road, Jaipur, Metro Station, Jaipur]"
9,9,16,"[social, official]","[Airport Road, Jaipur, Bus Stand, Jaipur, Univ..."


In [114]:
def calculate_credibility(message_count, sources):
    sources = set(sources)

    # Strong confirmation
    if message_count >= 3 and ("official" in sources or "emergency" in sources):
        return "High"

    # News + social but no official confirmation → suspicious
    if "news" in sources and "official" not in sources and "emergency" not in sources:
        return "Low"

    # Multiple reports but only public sources
    if message_count >= 2:
        return "Medium"

    return "Low"



In [115]:
event_summary["credibility"] = event_summary.apply(
    lambda row: calculate_credibility(row["message_count"], row["sources"]),
    axis=1
)

event_summary


,event_id,message_count,sources,locations,credibility
0,0,28,"[news, social, emergency, official]","[Airport Road, Jaipur, Railway Station, Jaipur...",High
1,1,26,"[social, official, emergency]","[Airport Road, Jaipur, Bus Stand, Jaipur, Rail...",High
2,2,44,"[social, emergency, official]","[Airport Road, Jaipur, Bus Stand, Jaipur, Rail...",High
3,3,16,"[social, official]","[Metro Station, Jaipur, Railway Station, Jaipu...",High
4,4,10,"[social, official]","[Airport Road, Jaipur, Bus Stand, Jaipur, Metr...",High
5,5,12,"[news, social, official]","[University Road, Jaipur]",High
6,6,7,[social],"[Metro Station, Jaipur, Bus Stand, Jaipur, Vai...",Medium
7,7,13,[official],"[Bus Stand, Jaipur, Mansarovar, Jaipur, Univer...",High
8,8,5,[official],"[Airport Road, Jaipur, Metro Station, Jaipur]",High
9,9,16,"[social, official]","[Airport Road, Jaipur, Bus Stand, Jaipur, Univ...",High


In [116]:
def detect_risk_level(texts, locations, message_count):
    text_combined = " ".join(texts).lower()
    locations = " ".join(locations).lower()

    # High-risk keywords & places
    high_risk_keywords = ["fire", "blast", "explosion", "smoke"]
    sensitive_places = ["railway", "bus stand", "station"]

    # Medium-risk keywords & places
    medium_risk_keywords = ["crowd", "gathering", "people"]
    public_places = ["mall", "market"]

    # High risk conditions
    if any(word in text_combined for word in high_risk_keywords):
        return "High"
    if any(place in locations for place in sensitive_places):
        return "High"

    # Medium risk conditions
    if any(word in text_combined for word in medium_risk_keywords):
        return "Medium"
    if any(place in locations for place in public_places):
        return "Medium"

    # Low risk otherwise
    return "Low"


In [117]:
# Collect texts per event
event_texts = (
    events_df
    .groupby("event_id")["clean_text"]
    .apply(list)
    .reset_index(name="texts")
)

# Merge texts into event_summary
event_summary = event_summary.merge(event_texts, on="event_id")


In [118]:
event_summary["risk"] = event_summary.apply(
    lambda row: detect_risk_level(
        row["texts"],
        row["locations"],
        row["message_count"]
    ),
    axis=1
)

event_summary[["event_id", "credibility", "risk"]]


,event_id,credibility,risk
0,0,High,High
1,1,High,High
2,2,High,High
3,3,High,High
4,4,High,High
5,5,High,Medium
6,6,Medium,High
7,7,High,High
8,8,High,High
9,9,High,High


In [119]:
def make_decision(credibility, risk):
    # High risk cases
    if risk == "High":
        if credibility in ["High", "Medium"]:
            return "ALERT"
        else:
            return "MONITOR"

    # Medium risk cases
    if risk == "Medium":
        if credibility == "High":
            return "MONITOR"
        else:
            return "MONITOR"

    # Low risk cases
    return "IGNORE"


In [120]:
event_summary["decision"] = event_summary.apply(
    lambda row: make_decision(row["credibility"], row["risk"]),
    axis=1
)

event_summary[["event_id", "credibility", "risk", "decision"]]



,event_id,credibility,risk,decision
0,0,High,High,ALERT
1,1,High,High,ALERT
2,2,High,High,ALERT
3,3,High,High,ALERT
4,4,High,High,ALERT
5,5,High,Medium,MONITOR
6,6,Medium,High,ALERT
7,7,High,High,ALERT
8,8,High,High,ALERT
9,9,High,High,ALERT


In [121]:
def credibility_score(message_count, sources):
    score = 0
    sources = set(sources)

    if "official" in sources:
        score += 40
    if "emergency" in sources:
        score += 30
    if "news" in sources:
        score += 10

    score += min(message_count * 5, 30)

    return min(score, 100)


In [122]:
def risk_score(risk_level):
    if risk_level == "High":
        return 85
    elif risk_level == "Medium":
        return 55
    else:
        return 25


In [123]:
event_summary["credibility_score"] = event_summary.apply(
    lambda row: credibility_score(row["message_count"], row["sources"]),
    axis=1
)

event_summary["risk_score"] = event_summary["risk"].apply(risk_score)


In [124]:
def build_event_timeline(event_messages):
    timeline = []

    for idx, row in event_messages.iterrows():
        stage = "Detected"

        if row["source"] in ["official", "emergency"]:
            stage = "Confirmed"
        elif idx > 0:
            stage = "Developing"

        timeline.append({
            "time": row["time"],
            "stage": stage,
            "source": row["source"]
        })

    return timeline


In [125]:
timelines = {}

for event_id in events_df["event_id"].unique():
    event_msgs = events_df[events_df["event_id"] == event_id].sort_values("time")
    timelines[event_id] = build_event_timeline(event_msgs)


In [126]:
event_summary["timeline"] = event_summary["event_id"].map(timelines)


In [127]:
final_output = event_summary[[
    "event_id",
    "locations",
    "message_count",
    "sources",
    "credibility",
    "credibility_score",
    "risk",
    "risk_score",
    "decision",
    "timeline"
]]

final_output


,event_id,locations,message_count,sources,credibility,credibility_score,risk,risk_score,decision,timeline
0,0,"[Airport Road, Jaipur, Railway Station, Jaipur...",28,"[news, social, emergency, official]",High,100,High,85,ALERT,"[{'time': '10:25:00 AM', 'stage': 'Developing'..."
1,1,"[Airport Road, Jaipur, Bus Stand, Jaipur, Rail...",26,"[social, official, emergency]",High,100,High,85,ALERT,"[{'time': '10:35:00 AM', 'stage': 'Developing'..."
2,2,"[Airport Road, Jaipur, Bus Stand, Jaipur, Rail...",44,"[social, emergency, official]",High,100,High,85,ALERT,"[{'time': '10:55:00 AM', 'stage': 'Developing'..."
3,3,"[Metro Station, Jaipur, Railway Station, Jaipu...",16,"[social, official]",High,70,High,85,ALERT,"[{'time': '11:05:00 AM', 'stage': 'Developing'..."
4,4,"[Airport Road, Jaipur, Bus Stand, Jaipur, Metr...",10,"[social, official]",High,70,High,85,ALERT,"[{'time': '10:00:00 AM', 'stage': 'Confirmed',..."
5,5,"[University Road, Jaipur]",12,"[news, social, official]",High,80,Medium,55,MONITOR,"[{'time': '10:05:00 AM', 'stage': 'Developing'..."
6,6,"[Metro Station, Jaipur, Bus Stand, Jaipur, Vai...",7,[social],Medium,30,High,85,ALERT,"[{'time': '10:15:00 AM', 'stage': 'Developing'..."
7,7,"[Bus Stand, Jaipur, Mansarovar, Jaipur, Univer...",13,[official],High,70,High,85,ALERT,"[{'time': '10:20:00 AM', 'stage': 'Confirmed',..."
8,8,"[Airport Road, Jaipur, Metro Station, Jaipur]",5,[official],High,65,High,85,ALERT,"[{'time': '10:30:00 AM', 'stage': 'Confirmed',..."
9,9,"[Airport Road, Jaipur, Bus Stand, Jaipur, Univ...",16,"[social, official]",High,70,High,85,ALERT,"[{'time': '10:45:00 AM', 'stage': 'Developing'..."


In [128]:
# Convert dataframe to JSON
final_json = final_output.to_dict(orient="records")

final_json



[{'event_id': 0,
  'locations': ['Airport Road, Jaipur',
   'Railway Station, Jaipur',
   'Bus Stand, Jaipur',
   'Metro Station, Jaipur',
   'City Mall, Jaipur',
   'Civil Lines, Jaipur',
   'Vaishali Nagar, Jaipur'],
  'message_count': 28,
  'sources': ['news', 'social', 'emergency', 'official'],
  'credibility': 'High',
  'credibility_score': 100,
  'risk': 'High',
  'risk_score': 85,
  'decision': 'ALERT',
  'timeline': [{'time': '10:25:00 AM',
    'stage': 'Developing',
    'source': 'social'},
   {'time': '10:25:00 PM', 'stage': 'Developing', 'source': 'social'},
   {'time': '10:30:00 PM', 'stage': 'Confirmed', 'source': 'official'},
   {'time': '11:25:00 AM', 'stage': 'Developing', 'source': 'social'},
   {'time': '11:25:00 PM', 'stage': 'Developing', 'source': 'social'},
   {'time': '11:30:00 AM', 'stage': 'Developing', 'source': 'news'},
   {'time': '11:30:00 PM', 'stage': 'Confirmed', 'source': 'official'},
   {'time': '12:15:00 AM', 'stage': 'Developing', 'source': 'social'}

In [129]:
import json

with open("final_events.json", "w") as f:
    json.dump(final_json, f, indent=4)

print("final_events.json saved successfully")


final_events.json saved successfully
